<a href="https://colab.research.google.com/github/John1495/RNA-3D/blob/main/mODEL_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyngrok
!pip install mlflow torch_geometric

  Using cached mlflow-2.22.0-py3-none-any.whl.metadata (30 kB)
  Using cached mlflow_skinny-2.22.0-py3-none-any.whl.metadata (31 kB)
  Using cached flask-3.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached alembic-1.15.2-py3-none-any.whl.metadata (7.3 kB)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached graphene-3.4.3-py2.py3-none-any.whl.metadata (6.9 kB)
  Using cached databricks_sdk-0.51.0-py3-none-any.whl.metadata (39 kB)
  Using cached fastapi-0.115.12-py3-none-any.whl.metadata (27 kB)
  Using cached GitPython-3.1.44-py3-none-any.whl.metadata (13 kB)
  Using cached opentelemetry_api-1.32.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached opentelemetry_sdk-1.32.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached starlette-0.46.2-py3-none-any.whl.metadata (6.2 kB)
  Using cached opentelemetry_semantic_conventions-0.53b1-py3-none-any.whl.metadata (2.5 kB)
Using cached mlflow-2.22.0-py3-n

In [ ]:
 !pip install biopython viennarna torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 124.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 53.4 MB/s eta 0:00:00


In [ ]:
# First install MLflow and required dependencies
!pip install mlflow

# Install PyTorch Geometric and its dependencies
!pip install torch_geometric

# Install pyngrok for MLflow UI access
!pip install pyngrok

# Sometimes you need to explicitly install these for torch_geometric
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-{torch.__version__}.html

  Using cached mlflow-2.22.0-py3-none-any.whl.metadata (30 kB)
  Using cached mlflow_skinny-2.22.0-py3-none-any.whl.metadata (31 kB)
  Using cached flask-3.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached alembic-1.15.2-py3-none-any.whl.metadata (7.3 kB)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached graphene-3.4.3-py2.py3-none-any.whl.metadata (6.9 kB)
  Using cached databricks_sdk-0.51.0-py3-none-any.whl.metadata (39 kB)
  Using cached fastapi-0.115.12-py3-none-any.whl.metadata (27 kB)
  Using cached GitPython-3.1.44-py3-none-any.whl.metadata (13 kB)
  Using cached opentelemetry_api-1.32.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached opentelemetry_sdk-1.32.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached starlette-0.46.2-py3-none-any.whl.metadata (6.2 kB)
  Using cached opentelemetry_semantic_conventions-0.53b1-py3-none-any.whl.metadata (2.5 kB)
Using cached mlflow-2.22.0-py3-n

In [ ]:
import mlflow
print("MLflow version:", mlflow.__version__)

import torch
print("PyTorch version:", torch.__version__)

import torch_geometric
print("PyTorch Geometric version:", torch_geometric.__version__)

ModuleNotFoundError: No module named 'mlflow'

In [ ]:
import pandas as pd

# Load CSVs
df_sequences = pd.read_csv("/kaggle/cleaned_train_sequences2.csv")
df_labels = pd.read_csv("/kaggle/train_labels1.csv")

# Extract 'target_id' from label 'ID' column
df_labels['target_id'] = df_labels['ID'].apply(lambda x: "_".join(x.split("_")[:2]))

# Merge on 'target_id'
df_merged = df_labels.merge(df_sequences, on='target_id', how='left')

# Pick a sample target_id to inspect (e.g., first one)
sample_id = df_merged['target_id'].unique()[0]

# Extract data for that sample
df_sample = df_merged[df_merged['target_id'] == sample_id].copy()

# Sort by residue ID for clarity
df_sample.sort_values(by='resid', inplace=True)

# Display result
print(f"Sample RNA ID: {sample_id}")
print(df_sample[['resid', 'resname', 'x_1', 'y_1', 'z_1']])


Sample RNA ID: 17RA_A
    resid resname        x_1        y_1     z_1
0       1       G  35.856998 -10.769000  -7.548
11      2       G  30.230000 -12.075000  -8.614
14      3       C  23.968000 -11.356000  -7.690
15      4       G  19.296000  -9.874000  -4.778
16      5       U  16.362000  -6.047000  -0.706
17      6       A  15.636000  -1.549000   2.463
18      7       A  16.969999   2.893000   4.626
19      8       G  20.391001   6.862000   5.549
20      9       G  24.370001   9.630000   3.348
1      10       A  26.341999  12.365000  -0.594
2      11       U  23.917999  16.023001  -5.418
3      12       U  24.938000  15.565000 -11.243
4      13       A  25.587999  10.095000 -10.004
5      14       C  28.333000   7.804000  -6.255
6      15       C  28.948999   4.836000  -0.756
7      16       U  26.749001   1.390000   3.287
8      17       A  24.118999  -2.821000   6.026
9      18       U  22.771000  -7.665000   5.355
10     19       G  22.330000 -13.626000   3.107
12     20       C 

In [ ]:
import pandas as pd

# Data Loading & Merging
df_seq = pd.read_csv("/kaggle/cleaned_train_sequences2.csv")
df_lbl = pd.read_csv("/kaggle/train_labels1.csv")
df_lbl['target_id'] = df_lbl['ID'].str.extract(r'^([^_]+_[^_]+)')  # More efficient extraction
df = df_lbl.merge(df_seq, on='target_id', how='left')

# Sample Inspection
sample_id = df['target_id'].iloc[0]  # First sample
sample_data = (df[df['target_id'] == sample_id]
               .sort_values('resid')
               [['resid', 'resname', 'x_1', 'y_1', 'z_1']])

print(f"Sample RNA ID: {sample_id}\n{sample_data.head()}")

Sample RNA ID: 17RA_A
    resid resname        x_1     y_1    z_1
0       1       G  35.856998 -10.769 -7.548
11      2       G  30.230000 -12.075 -8.614
14      3       C  23.968000 -11.356 -7.690
15      4       G  19.296000  -9.874 -4.778
16      5       U  16.362000  -6.047 -0.706


In [ ]:
import torch
from torch_geometric.data import Data

def create_graph(group):
    """Convert RNA residues into graph nodes with spatial edges"""
    coords = group[['x_1', 'y_1', 'z_1']].values
    features = group[['residue_encoded', 'planarity', 'mean_5A_dist']].values

    # Create k-NN graph (20 nearest neighbors within 10Å cutoff)
    dist_matrix = squareform(pdist(coords))
    edge_index = []
    edge_attr = []

    for i in range(len(coords)):
        neighbors = np.where((dist_matrix[i] < 10) & (dist_matrix[i] > 0))[0]
        top_k = min(20, len(neighbors))
        if top_k > 0:
            closest = np.argpartition(dist_matrix[i, neighbors], top_k)[:top_k]
            for j in neighbors[closest]:
                edge_index.append([i, j])
                edge_attr.append([dist_matrix[i,j]])

    return Data(
        x=torch.FloatTensor(features),
        edge_index=torch.LongTensor(np.array(edge_index)).T,
        edge_attr=torch.FloatTensor(edge_attr),
        pos=torch.FloatTensor(coords)
    )

In [ ]:
from torch_geometric.nn import MessagePassing
from torch.nn import Sequential, Linear, ReLU

class RNAGNN(MessagePassing):
    def __init__(self, node_dim=16, edge_dim=4):
        super().__init__(aggr='mean')
        self.edge_mlp = Sequential(
            Linear(1, edge_dim),
            ReLU(),
            Linear(edge_dim, edge_dim)
        )
        self.node_mlp = Sequential(
            Linear(node_dim + edge_dim, node_dim),
            ReLU(),
            Linear(node_dim, node_dim)
        )

    def forward(self, x, edge_index, edge_attr):
        return self.propagate(edge_index, x=x, edge_attr=edge_attr)

    def message(self, x_j, edge_attr):
        edge_features = self.edge_mlp(edge_attr)
        return torch.cat([x_j, edge_features], dim=-1)

    def update(self, aggr_out):
        return self.node_mlp(aggr_out)

In [ ]:
class RNA3DPredictor(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = Linear(3, 16)  # Input feature dimension
        self.gnn_layers = torch.nn.ModuleList(
            [RNAGNN() for _ in range(3)]
        )
        self.decoder = Sequential(
            Linear(16, 32),
            ReLU(),
            Linear(32, 3)  # Predict x,y,z coordinates
        )

    def forward(self, data):
        x = self.encoder(data.x)
        for layer in self.gnn_layers:
            x = layer(x, data.edge_index, data.edge_attr)
        return self.decoder(x)

In [ ]:
def tm_score_loss(pred_pos, true_pos):
    """Differentiable TM-Score approximation"""
    d = torch.norm(pred_pos - true_pos, dim=-1)
    d0 = 1.24 * (len(pred_pos) - 15)**(1/3) - 1.8
    weights = 1 / (1 + (d/d0)**2)
    return -weights.mean()  # Maximize TM-Score

In [ ]:
def train(model, graphs, epochs=100):
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for graph in graphs:
            optimizer.zero_grad()
            pred = model(graph)
            loss = tm_score_loss(pred, graph.pos)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        scheduler.step(total_loss)
        print(f"Epoch {epoch}: Loss {total_loss/len(graphs):.4f}")

In [ ]:
class RNA3DPredictor(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # Encoder
        self.encoder = Linear(3, 16)

        # GNN Layers with regularization
        self.gnn_layers = torch.nn.ModuleList(
            [RNAGNN() for _ in range(3)]
        )

        # Regularization components
        self.dropout = torch.nn.Dropout(0.3)
        self.layer_norm = torch.nn.LayerNorm(16)

        # Decoder
        self.decoder = Sequential(
            Linear(16, 32),
            ReLU(),
            Linear(32, 3)
        )

    def forward(self, data):
        x = self.encoder(data.x)
        for layer in self.gnn_layers:
            x = layer(x, data.edge_index, data.edge_attr)
            # Apply regularization
            x = self.layer_norm(x)
            x = self.dropout(x)
        return self.decoder(x)

In [ ]:
def forward(self, data):
    x = self.encoder(data.x)
    for layer in self.gnn_layers:
        residual = x
        x = layer(x, data.edge_index, data.edge_attr)
        x = self.layer_norm(x + residual)  # Skip connection
        x = self.dropout(x)
    return self.decoder(x)

In [ ]:
from matplotlib import pyplot as plt

def visualize_prediction(true, pred):
    fig = plt.figure(figsize=(10,5))
    ax1 = fig.add_subplot(121, projection='3d')
    ax1.scatter(*true.T, c='blue', label='True')
    ax2 = fig.add_subplot(122, projection='3d')
    ax2.scatter(*pred.T, c='red', label='Predicted')
    plt.show()

In [ ]:
class AttentionRNAGNN(MessagePassing):
    def __init__(self, node_dim=32):
        super().__init__(aggr='add')
        self.attention_mlp = Sequential(
            Linear(node_dim * 2, node_dim),
            ReLU(),
            Linear(node_dim, 1)
        )
        self.node_update = Sequential(
            Linear(node_dim * 2, node_dim),
            LayerNorm(node_dim),
            ReLU()
        )

    def forward(self, x, edge_index):
        return self.propagate(edge_index, x=x)

    def message(self, x_i, x_j):
        # Compute attention weights
        alpha = torch.cat([x_i, x_j], dim=-1)
        alpha = self.attention_mlp(alpha)
        alpha = softmax(alpha, index=self.edge_index[0], dim=0)
        return alpha * x_j

    def update(self, aggr_out, x):
        return self.node_update(torch.cat([x, aggr_out], dim=-1))

In [ ]:
class RNAEnsemble(torch.nn.Module):
    def __init__(self, num_models=5):
        super().__init__()
        self.models = ModuleList([
            RNA3DPredictor(use_attention=True) if i%2 else RNA3DPredictor(use_attention=False)
            for i in range(num_models)
        ])
        self.consensus = Linear(num_models * 3, 3)

    def forward(self, data):
        preds = [model(data) for model in self.models]
        return self.consensus(torch.cat(preds, dim=-1))

In [47]:
def validate(model, val_graphs):
    model.eval()
    tm_scores = []

    with torch.no_grad():
        for graph in val_graphs:
            pred = model(graph)
            # Calculate TM-score components
            d = torch.norm(pred - graph.pos, dim=1)
            L = len(graph.pos)
            d0 = 1.24 * (L - 15)**(1/3) - 1.8
            w = 1 / (1 + (d/d0)**2)
            tm = (1/L) * w.sum().item()

            tm_scores.append(tm)

            # Visualization every 10th sample
            if len(tm_scores) % 10 == 0:
                visualize_comparison(graph.pos.cpu().numpy(),
                                   pred.cpu().numpy(),
                                   title=f"TM-score: {tm:.3f}")

    avg_tm = np.mean(tm_scores)
    print(f"Validation TM-score: {avg_tm:.4f} ± {np.std(tm_scores):.4f}")
    return avg_tm